## Modules Importing
Import all necessary modules and add NCVX src folder to system path.

In [1]:
import time
import numpy as np
import torch
import numpy.linalg as la
from scipy.stats import norm
import sys
## Adding NCVX directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/NCVX')
from ncvx import ncvx
from ncvxStruct import Options, GeneralStruct 
from sklearn.datasets import make_sparse_coded_signal
from sklearn.decomposition import DictionaryLearning

## Data Generation 
Specify torch device, and generate data

Use GPU for this problem. If no cuda device available, please set *device = torch.device('cpu')*

In [2]:
device = torch.device('cuda')
n_features = 10
n_samples = 20
n_components = 8
n_nonzero_coefs = 5

X, dictionary, code = make_sparse_coded_signal(
    n_samples=n_samples, n_components=n_components, n_features=n_features, n_nonzero_coefs=n_nonzero_coefs,
    random_state=6)

X = torch.from_numpy(X).to(device=device, dtype=torch.double)
# code = torch.from_numpy(code).to(device=device, dtype=torch.double)

In [3]:
# ce = torch.sum(code**2,dim=0)
# print(ce)
# ce1 = torch.norm(code,p=2,dim=0)**2
# ce1

## Problem Definition

Specify optimization variables, and objective and constraint(s).

Note: please strictly follow the format of comb_fn, which will be used in the NCVX main algortihm.

In [4]:
# variables and corresponding dimensions.
var_in = {"U": [n_features,n_components], "V": [n_components, n_samples]}
alpha = 1.0 # Sparsity controlling parameter.

def comb_fn(X_struct):
    U = X_struct.U
    V = X_struct.V
    U.requires_grad_(True)
    V.requires_grad_(True)
    
    # objective function
#     f = 0.5 * torch.norm(X-U@V, p='fro') + alpha * torch.norm(U,p=1)
    f = 0.5 * torch.norm(X-U@V, p='fro') + alpha * torch.sum(torch.abs(U))

    # inequality constraint, matrix form
    ci = None

    # equality constraint 
    ce = GeneralStruct()
#     ce.c1 = torch.norm(V,p=2,dim=0)-1
    # torch norm is less efficient
    ce.c1 = torch.sum(V**2,dim=0) - 1

    return [f,ci,ce]


## User Options
Specify user-defined options for NCVX

In [5]:
opts = Options()
opts.QPsolver = 'osqp' 
opts.maxit = 1000
np.random.seed(1)
x0 = norm.ppf(np.random.rand((n_features+n_samples)*n_components,1))
x0 /= la.norm(x0,2)
opts.x0 = torch.from_numpy(x0).to(device=device, dtype=torch.double)
opts.print_frequency = 10
opts.opt_tol = 1e-6

## Main Algorithm

In [6]:
start = time.time()
soln = ncvx(combinedFunction = comb_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))
print(max(abs(soln.final.x))) # should be close to 1



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════╗
║  NCVX requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.              ║
║  To disable this notice, set opts.quadprog_info_msg = False                               ║
╚═══════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
NCVX: A User-Friendly and Scalable Package for Nonconvex Optimization in Machine Learning                        ║ 
Version 1.1.1                                                                                                    ║ 
MIT License Copyright (c) 2021 SUN Group @ UMN                                                                   ║ 
═════════════════════════════════════════════════════════════════